<a href="https://colab.research.google.com/github/mikad98/mastersthesis_ibd_wearable/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [2]:
import warnings
import pandas as pd
import numpy as np
import ast
import re
from datetime import date
from scipy.stats import linregress
from google.colab import drive
drive.mount('/content/drive')
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

Mounted at /content/drive


### Data Preprocessing - Subjective

In [ ]:
# year_of_birth to age
def year_to_age(year_of_birth):
    current_year = date.today().year
    try:
        year = float(year_of_birth)
        if year <= 0 or year > current_year:
            return None
        return int(current_year - year)
    except (ValueError, TypeError):
        return None

# mappings
chronic_flare_mapping = {
        'vacantForMoreThanOneYear': 0,
        'oneToThree': 1,
        'threeToSix': 2,
        'sixToNine': 3,
        'moreThanNine': 4,
        'chronicFlares': 5,
    }

activity_mapping = {
        'zero': 0,
        'below30min': 1,
        'below1h': 2,
        'below2h': 3,
        'below4h': 4,
        'below8h': 5,
        'above8h': 6
    }

diagnosis_mapping = {
        "colitisUlcerosa": "UC",
        "crohnsDisease": "CD",
        "Crohn's disease": "CD",
    }

gender_mapping = {
        "female": "F",
        "male": "M"
    }

alcohol_mapping = {
        "Yes": 1,
        "A little": 1,
        "No": 0,
        "Unsure": np.nan
    }

period_mapping = {
        "Yes": 1,
        "Unsure": 0,
        "False": np.nan,
        "No": 0,
    }

# Load Data
subjective = pd.read_csv('/content/drive/My Drive/coreway_ml/Thesis - Mika/Data/raw_subjective_data_2025-09-26.csv')

# Convert to datetime and normalize (remove time part)
subjective['date'] = pd.to_datetime(subjective['date'], format='mixed').dt.normalize()

# Convert "yearOfBirth" into "age"
subjective['age'] = subjective['yearOfBirth'].apply(year_to_age)

# Store provider as strings
subjective["connectedWearableName"] = subjective["connectedWearableName"].apply(lambda x: (np.nan if (x == "" or x == "[]") else ", ".join(ast.literal_eval(x)) if isinstance(x, str) and x.startswith("[") else x))

# Replace non-physiological values with NaNs
subjective["hrvMeasurement"] = subjective["hrvMeasurement"].where(subjective["hrvMeasurement"].between(5, 300), np.nan)

# Apply mapping's
subjective['hasConsumedAlcoholInLast24Hours'] = subjective['hasConsumedAlcoholInLast24Hours'].map(alcohol_mapping)
subjective['yearly_flares'] = subjective['flaresPerYear'].map(chronic_flare_mapping)
subjective['activity_dur'] = subjective['physicalEffort'].map(activity_mapping)
subjective['diagnosis'] = subjective['diagnosis'].map(diagnosis_mapping)
subjective['isOnPeriod'] = subjective['isOnPeriod'].map(period_mapping)
subjective['gender'] = subjective['gender'].map(gender_mapping)

# Drop row's for underrepresented diagnoses/gender
subjective.dropna(subset=["gender", "diagnosis"], inplace=True)

subjective.drop(['hasOtherHealthProblems',
                 'stressType',
                 'symptoms',
                 'otherSymptoms',
                 'otherHealthProblems',
                 'mentalStressLevel',
                 'notes',
                 'generalCondition',
                 'complications',
                 'urgencyOfDefecation',
                 'bloodInStool',
                 'stomachPain',
                 'abdominalResistance',
                 'terraUserId',
                 'additionalIllnesses',
                 'otherKnownCatalysts',
                 'hrvMeasurementMethod',
                 'backendSymptoms',
                 'medication',
                 'otherMedication',
                 'hrvMeasurementMethodName',
                 'hasAskedToConnectWearable',
                 'hasConnectedWearable',
                 'knownCatalysts',
                 'alcoholPortions',
                 'stoolsPerDay',
                 'stoolsPerNight',
                 'unformedStoolsPerDay',
                 'backendKnownCatalysts',
                 'hasStoma',
                 'otherAdditionalIllnesses',
                 'flaresPerYear',
                 'yearOfBirth',
                 'physicalEffort',
                 'diseaseRelapsesEvaluation'
                 ], axis=1, inplace=True)

# Renaming
subjective = subjective.rename(columns={
                 'userId': 'user_id',
                 'hrvMeasurement': 'hrv',
                 'sleepQualityDegree': 'sleep',
                 'stressLevelDegree': 'stress',
                 'physicalActivityExertionDegree': 'activity_deg',
                 'symptomDegree': 'symptom_deg',
                 'hasConsumedAlcoholInLast24Hours': 'alcohol_last_24h',
                 'connectedWearableName': 'provider',
                 'isOnPeriod': 'on_period',
                 'rateAsFlare': 'rate_as_flare',
                 'diagnosis_colitisUlcerosa': 'CU',
                 'gender_female': 'female',
                 'gender_male': 'male',
                 'flaresPerYear': 'flares_per_year'
                 })

# Reordering columns
subjective = subjective[[
                 'user_id',
                 'date',
                 'gender',
                 'age',
                 'diagnosis',
                 'alcohol_last_24h',
                 'on_period',
                 'hrv',
                 'provider',
                 'sleep',
                 'stress',
                 'activity_dur',
                 'activity_deg',
                 'symptom_deg',
                 'rate_as_flare',
                 'yearly_flares'
                 ]]

### Data Preprocessing - Objective

In [ ]:
def extract_hhmm(s):
    if pd.isna(s):
        return None
    # Match HH:MM at start of time part
    match = re.search(r'T(\d{2}:\d{2})', s)
    return match.group(1) if match else None

def compute_sleep_length(row):
    start_time = pd.to_datetime(row['start'], format='%H:%M')
    end_time = pd.to_datetime(row['end'], format='%H:%M')

    # Handle crossing midnight
    if end_time < start_time:
        end_time += pd.Timedelta(days=1)

    diff = end_time - start_time
    hours, remainder = divmod(diff.seconds, 3600)
    minutes = remainder // 60

    return f"{hours:02d}:{minutes:02d}"

def compute_std_rmssd(x):
    try:
        if isinstance(x, str):
            x = ast.literal_eval(x)
        if isinstance(x, list) and len(x) > 0:
            return np.std(x)
    except:
        pass
    return None

def compute_cv_rmssd(x):
    try:
        if isinstance(x, str):
            x = ast.literal_eval(x)
        if isinstance(x, list) and len(x) > 0:
            mean_val = np.mean(x)
            std_val = np.std(x)
            if mean_val != 0:
                return std_val / mean_val
    except:
        pass
    return None

def compute_min_rmssd(x):
    try:
        if isinstance(x, str):
            x = ast.literal_eval(x)
        if isinstance(x, list) and len(x) > 0:
            return np.min(x)
    except:
        pass
    return None

def compute_max_rmssd(x):
    try:
        if isinstance(x, str):
            x = ast.literal_eval(x)
        if isinstance(x, list) and len(x) > 0:
            return np.max(x)
    except:
        pass
    return None

def compute_slope_rmssd(x):
    try:
        if isinstance(x, str):
            x = ast.literal_eval(x)
        if isinstance(x, list) and len(x) > 1:
            time_indices = np.arange(len(x))
            slope, _, _, _, _ = linregress(time_indices, x)
            return slope
    except:
        pass
    return None

# Load Data
raw_summary_wearable_data = pd.read_csv('/content/drive/My Drive/coreway_ml/Thesis - Mika/Data/raw_summary_wearable_data_2025-09-26.csv')
raw_flattened_wearable_data = pd.read_csv('/content/drive/My Drive/coreway_ml/Thesis - Mika/Data/raw_flattened_wearable_data_2025-09-26.csv')

# Standardize date-column for merging
raw_summary_wearable_data['date'] = pd.to_datetime(raw_summary_wearable_data['date'].str.slice(0, 19))
raw_flattened_wearable_data['date'] = pd.to_datetime(raw_flattened_wearable_data['date'].str.slice(0, 19))

objective = pd.merge(raw_summary_wearable_data, raw_flattened_wearable_data, on=['userId', 'date'], how='inner')

# Ensures right date, and trim to YYYY-MM-DD
objective['date'] = objective['end_time']
objective['date'] = pd.to_datetime(objective['date'].str.slice(0, 10))

# Extract HH:MM
for col in ['start_time', 'end_time']: objective[col] = objective[col].apply(extract_hhmm)

# Drop columns - uninsteresting or lack of data
objective.drop([
    'terra_user_id',
    'provider_y',
    'sleep_score',
    'delta_temperature',
    'user_max_hr_bpm',
    'on_demand_reading',
    'breaths_start_time',
    'breaths_end_time',
    'max_breaths_per_min',
    'min_breaths_per_min',
    'duration_in_bed_seconds',
    'num_REM_events',
    'duration_long_interruption_seconds',
    'duration_short_interruption_seconds',
    'num_out_of_bed_events',
    'num_wakeup_events',
    'sleep_latency_seconds',
    'wake_up_latency_seconds',
    'max_hr_bpm',
    'min_hr_bpm',
    'bpm_array_length',
    'timestamp_intervals_seconds_hrv_rmssd',
    'hrv_rmssd_array_length',
    'level',
    'timestamp_intervals_seconds_level',
    'level_array_length',
    'percentage_array_length',
    'breaths_per_min_array_length',
    'timestamp_intervals_seconds_hrv_sdnn',
    'timestamp_intervals_seconds_bpm',
    'timestamp_intervals_seconds_percentage',
    'timestamp_intervals_seconds_breaths_per_min',
    'hrv_sdnn_array_length'
    ], axis=1, inplace=True)

# Renaming
objective = objective.rename(columns={
    'userId': 'user_id',
    'provider_x': 'provider',
    'start_time': 'start',
    'end_time': 'end',
    'avg_hr_bpm': 'avg_bpm',
    'resting_hr_bpm': 'rhr',
    'sleep_efficiency': 'sleep_eff',
    'breaths_per_min': 'breaths',
    'avg_breaths_per_min': 'avg_breaths',
    'avg_saturation_percentage': 'avg_SpO2',
    'duration_REM_sleep_state_seconds': 'dur_REM',
    'duration_asleep_state_seconds': 'dur_asleep',
    'duration_deep_sleep_state_seconds': 'dur_deep',
    'duration_light_sleep_state_seconds': 'dur_light',
    'duration_awake_state_seconds': 'dur_awake',
    'percentage': 'SpO2'
    })

# Infer sleep_length
objective['length'] = objective.apply(compute_sleep_length, axis=1)

# Remove duplicates & Aggregate information of the remaining duplicates & Only keep the longest sleep recordings per day (e.g. deletes mid-day nap's)
objective = objective.drop_duplicates()
objective = objective.groupby(['user_id', 'date', 'start', 'end'], as_index=False).agg(lambda x: x.dropna().iloc[0] if x.notna().any() else np.nan)
objective = objective.loc[objective.groupby(['user_id', 'date'])['length'].idxmax()].reset_index(drop=True)

# Convert durations from seconds to hours and compute sleep stage percentages
objective[['dur_asleep', 'dur_REM', 'dur_deep', 'dur_light', 'dur_awake']] /= 3600
objective[['REM_pct', 'deep_pct', 'light_pct']] = (objective[['dur_REM', 'dur_deep', 'dur_light']].div(objective['dur_asleep'], axis=0)) * 100
objective[['sleep_eff']] = objective[['sleep_eff']] * 100

# Drop rows, where sleep metrics are implausible
objective = objective[objective["dur_asleep"] >= 2]
objective = objective[objective["REM_pct"].between(0, 40) & objective["light_pct"].between(0, 90) & objective["deep_pct"].between(0, 40)]
total = objective["REM_pct"] + objective["light_pct"] + objective["deep_pct"]
objective = objective[total.between(90, 110)]
objective["sleep_eff"] = objective["sleep_eff"].where(objective["sleep_eff"].between(25, 100), np.nan)

# Replace non-physiological values with NaNs
objective["avg_hrv_sdnn"] = objective["avg_hrv_sdnn"].where(objective["avg_hrv_sdnn"].between(5, 300), np.nan)
objective["avg_hrv_rmssd"] = objective["avg_hrv_rmssd"].where(objective["avg_hrv_rmssd"].between(5, 300), np.nan)
objective["avg_SpO2"] = objective["avg_SpO2"].where(objective["avg_SpO2"].between(85, 100), np.nan)
objective["avg_bpm"] = objective["avg_bpm"].where(objective["avg_bpm"].between(30, 150), np.nan)
objective["rhr"] = objective["rhr"].where(objective["rhr"].between(30, 150), np.nan)

# derive HRV features
objective['std_rmssd'] = objective['hrv_rmssd'].apply(compute_std_rmssd)
objective['cv_rmssd'] = objective['hrv_rmssd'].apply(compute_cv_rmssd)
objective['min_rmssd'] = objective['hrv_rmssd'].apply(compute_min_rmssd)
objective['max_rmssd'] = objective['hrv_rmssd'].apply(compute_max_rmssd)
objective['range_rmssd'] = objective['max_rmssd'] - objective['min_rmssd']
objective['slope_rmssd'] = objective['hrv_rmssd'].apply(compute_slope_rmssd)

# Reordering columns
objective = objective[[
    'user_id',
    'provider',
    'date',
    'start',
    'end',
    'length',
    'sleep_eff',
    'dur_asleep',
    'dur_REM',
    'REM_pct',
    'dur_deep',
    'deep_pct',
    'dur_light',
    'light_pct',
    'dur_awake',
    'hrv_rmssd',
    'avg_hrv_rmssd',
    'std_rmssd',
    'cv_rmssd',
    'min_rmssd',
    'max_rmssd',
    'range_rmssd',
    'slope_rmssd',
    'hrv_sdnn',
    'avg_hrv_sdnn',
    'bpm',
    'avg_bpm',
    'rhr',
    'SpO2',
    'avg_SpO2',
    'breaths',
    'avg_breaths'
    ]]

# Sorting
objective = objective.sort_values(by=['user_id', 'date', 'start', 'end']).reset_index(drop=True)

### Data Preprocessing - Merged

In [ ]:
merged = pd.merge(subjective, objective, on=['user_id', 'date'], how='inner')

# merge provider columns
merged["provider_y"] = merged["provider_y"].fillna(merged["provider_x"])
merged = merged.rename(columns={"provider_y": "provider"})
merged = merged.drop(columns=["provider_x"])

# Fill avg_hrv values if suitable (avg_sdnn for APPLE). Assumption: all others use RMSSD metric
mask_apple = merged["provider"] == "APPLE"
mask_else = merged["provider"] != "APPLE"
merged.loc[mask_apple, "avg_hrv_sdnn"] = merged.loc[mask_apple, "avg_hrv_sdnn"].fillna(merged.loc[mask_apple, "hrv"])
merged.loc[mask_else, "avg_hrv_rmssd"] = merged.loc[mask_else, "avg_hrv_rmssd"].fillna(merged.loc[mask_else, "hrv"])
merged = merged.drop(columns=["hrv"])

### Save Dataframes

In [ ]:
subjective.to_csv('/content/drive/My Drive/coreway_ml/Thesis - Mika/subjective.csv', index=False)
objective.to_csv('/content/drive/My Drive/coreway_ml/Thesis - Mika/objective.csv', index=False)
merged.to_csv('/content/drive/My Drive/coreway_ml/Thesis - Mika/merged.csv', index=False)